In [ ]:
from pygad import GA
from sumolib import checkBinary
import xml.etree.ElementTree as ET
import traci
import sys

In [ ]:
def simulate(program): # run simulation
    traci.start([checkBinary(program), "-c", "sumo/simulation.sumocfg", "--tripinfo-output", "sumo/tripinfo.xml"])
    while traci.simulation.getMinExpectedNumber() > 0: # traci simulating multiple clients
        traci.simulationStep()
    traci.close()
    sys.stdout.flush()
    
    xmldoc = ET.parse('sumo/tripinfo.xml')

    tripinfos = xmldoc.findall('tripinfo')

    waitingTime = 0
    for tripinfo in tripinfos:
        waitingTime += float(tripinfo.get('waitingTime'))
        
    return waitingTime # total waiting time of cars

In [ ]:
def updateLight(time):
    network = ET.parse('sumo/simulation.net.xml')
    signal = network.find('tlLogic')
    phases = signal.findall('phase')
    phases[0].set("duration", str(time))
    phases[4].set("duration", str(60.0-time))
    network.write("sumo/simulation.net.xml")

In [ ]:
default = 30.0
updateLight(default)
initialWait=simulate("sumo-gui")
waitTimes=[]
print(initialWait)

In [ ]:
def fitness(solution, solution_idx):
    updateLight(solution[0])
    waitingTime = simulate('sumo') # run simulation and get waiting times
    print("wait:"+str(waitingTime))
    waitTimes.append(waitingTime)
    return initialWait/waitingTime

In [ ]:
generations = 10
population = 10
parents_mating= 2
num_genes=1

init_range_low=0
init_range_high=60

parent_selection_type = "sss"
keep_parents = -1

mutation_type = "random"

crossover_type = "single_point"
ga = GA(fitness_func=fitness,
        num_generations=generations,
        sol_per_pop=population,
        num_parents_mating=parents_mating,
        num_genes=num_genes,
        init_range_low=init_range_low,
        init_range_high=init_range_high,
        parent_selection_type=parent_selection_type,
        keep_parents=keep_parents,
        mutation_type=mutation_type,
        crossover_type=crossover_type,
        mutation_num_genes=num_genes,
        save_solutions=True
)

In [ ]:
ga.run()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(waitTimes, label='Gaidīšanas laiks')
plt.legend()
plt.show()
ga.plot_fitness()
print("initial: "+str(initialWait)+" min: "+str(min(waitTimes)))
print(ga.population)